# What is Sklearn's HalvingGridSearchCV and HalvingRandomizedSearchCV?
## Even faster hyperparameter tuning for massive models
<img src='images/tune.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@karolina-grabowska?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Karolina Grabowska</a>
        on 
        <a href='https://www.pexels.com/photo/person-holding-tuning-pegs-4472108/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

### Successive Halving

### Loading and Preparing Data

### HalvingGridSearchCV

### Setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')